# Loading Dependencies

In [2]:
import warnings
import pkg_resources
warnings.filterwarnings('ignore')
pkg_resources.require("scipy==1.8")
pkg_resources.require("networkx==2.7")
import scipy
import networkx as nx

Mounted at /content/drive


In [3]:
import pickle
from src.models import *
from src.utils import *
from src.utils_data import *
import multiprocess
import argparse
import time
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import numpy as np
import networkx as nx
import scipy
import pickle
from functools import partial
import random
from random import randint

# Generating Data

In [6]:
graph_size = 10

In [8]:
main_data = dict()

weight = random.choice(['gaussian', 'uniform', 'lognormal'])
temp_data = generate_ER_parallel(num_samples=128, num_signals=4000, num_nodes=graph_size,
                                                         graph_hyper={'p':0.5}, weighted=weight, weight_scale=True)
main_data['z'] = temp_data['z']
main_data['W'] = temp_data['W']

x = 1375
Ps_lower, Ps_upper, scale = 0.1, 1, 1/2
Ps = scipy.stats.truncexpon(b=(Ps_upper-Ps_lower)/scale, loc=Ps_lower, scale=scale).rvs(x)

for p in Ps:
    weight = random.choice(['gaussian', 'uniform', 'lognormal'])
    temp_data = generate_ER_parallel(num_samples=32, num_signals=4000, num_nodes=graph_size,
                                                            graph_hyper={'p':p}, weighted=weight, weight_scale=True)
    main_data['z'] += temp_data['z']
    main_data['W'] += temp_data['W']

In [9]:
len(main_data['z'])

44128

# Saving the Dataset

In [10]:
with open('Saved Datasets/dataset_TrainingSet_{}nodes.pickle'.format(graph_size), 'wb') as handle:
    pickle.dump(main_data, handle, protocol=4)